# [Bert-base-Chinese 微调](https://blog.csdn.net/qq_43668800/article/details/131921617)

In [1]:
import torch
from torch.optim import AdamW
from datasets import load_dataset
from transformers import BertModel, BertTokenizer

## 模型

In [2]:
# 优先使用 GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device=', device)

device= cuda


In [3]:
# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')
# 需要移动到cuda上
pretrained.to(device)

# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 加载字典和分词工具
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, tokenizer.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(tokenizer.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.decoder(out.last_hidden_state[:, 15])
        return out


model = Model()
# 同样要移动到cuda
model.to(device)

Model(
  (decoder): Linear(in_features=768, out_features=21128, bias=True)
)

## 数据集

In [6]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)
        def f(data):
            return len(data['text']) > 30
        self.dataset = dataset.filter(f)
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, i):
        text = self.dataset[i]['text']
        return text


dataset = Dataset('train')

Using custom data configuration lansinuote--ChnSentiCorp-4d058ef86e3db8d5
Reusing dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1b28defac4421dc0.arrow


In [7]:
def collate_fn(data):
    # 编码
    data = tokenizer.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    # 把第15个词固定替换为mask
    labels = input_ids[:, 15].reshape(-1).clone().to(device)
    input_ids[:, 15] = tokenizer.get_vocab()[tokenizer.mask_token]
    # print(data['length'], data['length'].max())
    return input_ids, attention_mask, token_type_ids, labels


# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)
for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    break

print(len(loader))
print(tokenizer.decode(input_ids[0]))
print(tokenizer.decode(labels[0]))
print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)

574
[CLS] 电 池 续 航 能 力 标 称 4 小 时 ， 实 际 [MASK] 概 2. 5 至 3 小 时 。 机 器 有 时 [SEP]
大
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])


## 训练

In [8]:
import time, datetime

start_time = datetime.datetime.now()
print('Start time:', start_time)

# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()
model.train()
for epoch in range(5):
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 50 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            print(epoch, i, loss.item(), accuracy)

 
end_time = datetime.datetime.now()
print('End time:', end_time)

consume_time = end_time - start_time
print('Consume time of second:', consume_time.seconds)

Start time: 2024-03-17 06:43:12.718336
0 0 10.097411155700684 0.0
0 50 7.582120418548584 0.1875
0 100 6.687246322631836 0.125
0 150 6.288071632385254 0.125
0 200 5.54958438873291 0.25
0 250 5.036920070648193 0.25
0 300 4.433927059173584 0.375
0 350 4.677452564239502 0.375
0 400 4.231757640838623 0.5
0 450 4.016312599182129 0.3125
0 500 3.7853074073791504 0.4375
0 550 3.3336679935455322 0.6875
1 0 1.9432461261749268 0.6875
1 50 2.293134927749634 0.6875
1 100 3.0873708724975586 0.5625
1 150 1.9749011993408203 0.625
1 200 2.0814075469970703 0.4375
1 250 2.1085357666015625 0.5625
1 300 2.45772123336792 0.625
1 350 2.926966667175293 0.4375
1 400 2.176602840423584 0.5625
1 450 1.9999704360961914 0.6875
1 500 1.996634840965271 0.625
1 550 1.5553489923477173 0.6875
2 0 0.5573766827583313 0.875
2 50 0.7366800308227539 0.9375
2 100 1.9312366247177124 0.6875
2 150 0.9704812169075012 0.8125
2 200 1.15225088596344 0.875
2 250 0.6283098459243774 0.875
2 300 0.9777261018753052 0.75
2 350 1.1209981441

## 测试

In [9]:
# 测试
def test():
    model.eval()
    correct = 0
    total = 0
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        if i >= 10:
            break
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)
        print(i, tokenizer.decode(input_ids[0]))
        print('预测:', tokenizer.decode(out[0]), '，实际:', tokenizer.decode(labels[0]))
    print(correct / total)


test()

Using custom data configuration lansinuote--ChnSentiCorp-4d058ef86e3db8d5
Reusing dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-6b1a9471b2da2abf.arrow


0 [CLS] 屏 的 显 示 效 果 好 象 不 大 好 ， 不 知 [MASK] 是 才 拿 到 不 会 调 还 是 其 它 问 题 [SEP]
预测: 道 ，实际: 道
1 [CLS] [UNK] 屏 ， 亮 ， 可 视 角 度 大 （ 下 方 可 [MASK] 效 果 不 好 ） 。 较 轻 。 新 款 适 配 [SEP]
预测: 能 ，实际: 视
2 [CLS] 我 前 几 天 发 表 了 我 的 看 法 ， 竟 然 [MASK] 当 当 工 作 人 员 删 除 了 。 多 的 我 [SEP]
预测: 在 ，实际: 被
3 [CLS] 这 一 套 书 我 基 本 买 齐 了 ， 也 看 了 [MASK] 多 本 了 。 是 利 用 闲 暇 时 间 巩 固 [SEP]
预测: 很 ，实际: 好
4 [CLS] 配 置 不 错 ， 性 价 比 挺 高 的 。 重 量 [MASK] 很 轻 ， 自 带 [UNK] [UNK] 和 [UNK] [UNK] 两 种 驱 [SEP]
预测: 也 ，实际: 也
5 [CLS] 力 荐 《 白 蛇 》 这 个 故 事 ， 源 于 一 [MASK] 传 说 ， 白 蛇 为 女 ， 青 蛇 为 男 ， [SEP]
预测: 个 ，实际: 个
6 [CLS] 新 机 拿 到 手 就 有 硬 件 问 题 ， 而 且 [MASK] 了 6 天 才 到 货 ， 第 二 天 就 返 修 [SEP]
预测: 用 ，实际: 等
7 [CLS] 看 这 本 书 ， 让 我 想 起 我 们 那 些 远 [MASK] 的 流 年 。 那 时 候 我 们 每 个 人 都 [SEP]
预测: 远 ，实际: 去
8 [CLS] 没 有 附 赠 背 包 ， 而 且 usb 接 口 过 少 [MASK] 才 两 个 ， 共 用 口 室 不 能 接 usb 设 [SEP]
预测: ， ，实际: ，
9 [CLS] 请 协 成 不 要 再 介 绍 是 准 四 星 未 挂 [MASK] 的 该 酒 店 ， 最 好 不 要 再 与 这 家 [SEP]
预测: 机 ，实际: 牌
0.65


## 保存模型

In [10]:
import torch

model_path = '/root/workspace/model/model_bert-base-chinese/text_fill.pt'
# torch.save(model.state_dict(), model_path)
torch.save(model.state_dict(), model_path)
print('模型已保存至:', model_path)

模型已保存至: /root/workspace/model/model_bert-base-chinese/text_fill.pt


## 加载模型

In [11]:
my_model = Model()
my_model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [12]:
def predict(the_model, text_arr, mark_idx):
    the_model.eval()
    # 优先使用 GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('device=', device)
    the_model.to(device)

    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    torch.no_grad()
    x_encode = tokenizer.batch_encode_plus(
        # 传入的所有句子，有成对句子
        batch_text_or_text_pairs=text_arr,
        # 长度大于设置是否截断
        truncation=True,
        # 一律补齐，如果长度不够
        padding='max_length',
        add_special_tokens=True,
        max_length=30,
        # 可取值tf,pt,np,（tensorflow,pytorch,numpy）默认返回list
        return_tensors="pt",
        # 返回token_type_ids,第一句与特殊符号是0，第二句是1
        return_token_type_ids=True,
        # 返回attention_mask，填充是0，其他是1
        return_attention_mask=True,
        # 返回special_tokens_mask特殊符号标识，特殊是1，其他是0
        return_special_tokens_mask=True,
        # 返回长度,这里的长度是真实长度，而非设置的长度30了
        return_length=True
    )

    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = x_encode['input_ids']
    attention_mask = x_encode['attention_mask']
    token_type_ids = x_encode['token_type_ids']
    # 把指定词固定替换为mask
    input_ids[:, mark_idx] = tokenizer.get_vocab()[tokenizer.mask_token]
    
    y = the_model(input_ids=input_ids.to(device),
                  attention_mask=attention_mask.to(device),
                  token_type_ids=token_type_ids.to(device))
    res = y.argmax(dim=1)
    
    for i, text in enumerate(text_arr):
        print(i, tokenizer.decode(input_ids[i]))
        print('预测:', tokenizer.decode(res[i]), '，实际:', text[mark_idx-1])
    
    return tokenizer.decode(res)


texts = ['安装好了，感觉很好。所有的驱动官方都提供，各快捷键的驱动', 
         '晕死，发表的评论竟然无法编辑了？？我想改几个错别字就不行',
         '还是连锁酒店靠谱，卫生条件非常不错，热水也很充足']
mark = 15
predict(my_model, texts, mark)

device= cuda
0 [CLS] 安 装 好 了 ， 感 觉 很 好 。 所 有 的 驱 [MASK] 官 方 都 提 供 ， 各 快 捷 键 的 驱 动 [SEP]
预测: 动 ，实际: 动
1 [CLS] 晕 死 ， 发 表 的 评 论 竟 然 无 法 编 辑 [MASK] ？ ？ 我 想 改 几 个 错 别 字 就 不 行 [SEP]
预测: 了 ，实际: 了
2 [CLS] 还 是 连 锁 酒 店 靠 谱 ， 卫 生 条 件 非 [MASK] 不 错 ， 热 水 也 很 充 足 [SEP] [PAD] [PAD] [PAD] [PAD]
预测: 常 ，实际: 常


'动 了 常'